In [48]:

import numpy as np
import pandas as pd

# from keras.models import Sequential
import tensorflow as tf
# from keras.layers import Dense,LSTM
import sqlite3
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelBinarizer

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
from tensorflow.keras.layers import LSTM, Activation, Dense


In [51]:
cnx = sqlite3.connect('/content/drive/My Drive/color_sequence/db.sqlite3')
df = pd.read_sql_query("SELECT * FROM application_all_data",cnx)
df[:100]

,number,color
0,24,Black
1,17,Black
2,33,Black
3,24,Black
4,22,Black
...,...,...
95,17,Black
96,25,Red
97,28,Black
98,37,Red


In [62]:

encoder = LabelBinarizer()
Color = df['color']
dataset = encoder.fit_transform(Color)
print(dataset)
#dataset = list(dataset)
print(len(dataset))

[[1 0 0]
 [1 0 0]
 [1 0 0]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]
500000


In [95]:
#Color = df['color']
# data = [[dataset[i+j] for i in range(1000)] for j in range(25000)]
# target = [dataset[i+10] for i in range(25000)]

seq_len = 5
data,target=[],[]
for i in range(0,len(dataset),seq_len):
  data.append(dataset[i:i+seq_len])
  try:
    target.append(dataset[i+seq_len])
  except:
    target.append(dataset[0])

Data = np.array(data)
Target = np.array(target)

x_train,x_test,y_train,y_test = train_test_split(Data,Target,test_size=0.2,random_state = 4)

In [96]:
x_train.shape,y_train.shape

((80000, 5, 3), (80000, 3))

In [97]:
model =  tf.keras.Sequential()
model.add(tf.keras.layers.LSTM((50),batch_input_shape = (None,5,3),return_sequences=False))
model.add(Activation('relu'))
model.add(Dense(3))
model.add(Activation('softmax'))

In [103]:
# opt = tf.keras.optimizers.Adam(learning_rate=0.001)
# opt = tf.keras.optimizers.RMSprop(learning_rate=0.001,momentum =0.6)

# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-2,
#     decay_steps=10000,
#     decay_rate=0.7)

opt = tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0.5)
model.compile(optimizer=opt,loss = 'categorical_cross_entropy',metrics=["accuracy"])

In [101]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 50)                10800     
_________________________________________________________________
activation_27 (Activation)   (None, 50)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 3)                 153       
_________________________________________________________________
activation_28 (Activation)   (None, 3)                 0         
Total params: 10,953
Trainable params: 10,953
Non-trainable params: 0
_________________________________________________________________


In [102]:
model.fit(x_train,y_train,epochs=20)

Epoch 1/20
2500/2500 [==============================] - 10s 4ms/step - loss: 0.2031 - accuracy: 0.4963
Epoch 2/20
2500/2500 [==============================] - 10s 4ms/step - loss: 0.1876 - accuracy: 0.4986
Epoch 3/20
2500/2500 [==============================] - 10s 4ms/step - loss: 0.1824 - accuracy: 0.4986
Epoch 4/20
2500/2500 [==============================] - 10s 4ms/step - loss: 0.1800 - accuracy: 0.4987
Epoch 5/20
2500/2500 [==============================] - 10s 4ms/step - loss: 0.1786 - accuracy: 0.4988
Epoch 6/20
2500/2500 [==============================] - 10s 4ms/step - loss: 0.1777 - accuracy: 0.4988
Epoch 7/20
2500/2500 [==============================] - 10s 4ms/step - loss: 0.1772 - accuracy: 0.4981
Epoch 8/20
2500/2500 [==============================] - 10s 4ms/step - loss: 0.1768 - accuracy: 0.5000
Epoch 9/20
2500/2500 [==============================] - 10s 4ms/step - loss: 0.1765 - accuracy: 0.5000
Epoch 10/20
2500/2500 [==============================] - 12s 5ms/step - l

In [ ]:
model.predict(x_test)

In [85]:
  results = model.predict(x_test)

results = results.tolist()
true = 0
false = 0
y_test = y_test.tolist()



for i in range(len(results)):
    ind = results[i].index(max(results[i]))
    for j in range(3):
        results[i][j] = 0
    results[i][ind] = 1
        
for i in range(len(results)):
    if results[i] == y_test[i]:
        true+=1
    elif results[i] != y_test[i]:
        false+=1
        
        
accuracy = true/(true +false)
accuracy

0.5104

In [ ]:
def check(num):
    if num in [2,4,6,8,10,11,13,15,17,20,22,24,26,28,29,31,33,35]:
        target = 'Black'
    elif num in [1,3,5,7,9,12,14,16,18,19,21,23,25,27,30,32,34,36]:
        target = 'Red'
    elif num == 0:
        target = 'Green'
        
    if target == 'Red':
        target_enc = [[0,0,1]]
    elif target == 'Black':
        target_enc = [[1,0,0]]
    elif target == 'Green':
        target_enc = [[0,1,0]]
    
    
    prediction = model.predict(np.array([Data[-1]]))
    prediction = prediction.tolist()
    
    ind = prediction[0].index(max(prediction[0]))
    for i in range(3):
        prediction[0][i] = 0
    prediction[0][ind] = 1

    if prediction == [[1,0,0]]:
        predicted_color = 'Black'
    elif prediction == [[0,1,0]]:
        predicted_color = 'Green'
    elif prediction == [[0,0,1]]:
        predicted_color = 'Red'
    
    iscorrect = (predicted_color ==  target) 
    print(iscorrect)
    return predicted_color
        

        


In [ ]:
# import numpy as np
# import pandas as pd

# from keras.models import Sequential

# from keras.layers import Dense,LSTM
# import sqlite3
# from sklearn.model_selection import train_test_split

# from sklearn.preprocessing import LabelBinarizer

# cnx = sqlite3.connect('db.sqlite3')
# df = pd.read_sql_query("SELECT * FROM application_all_data",cnx)

# encoder = LabelBinarizer()
# Color = df['color']
# dataset = encoder.fit_transform(Color)

# data = [[dataset[i+j] for i in range(20)] for j in range(25000)]
# target = [dataset[i+20] for i in range(25000)]
# Data = np.array(data)
# Target = np.array(target)

# x_train,x_test,y_train,y_test = train_test_split(Data,Target,test_size=0.9,random_state = 4)


# model =  Sequential()
# model.add(LSTM((3),batch_input_shape = (None,20,3),return_sequences = False))
# model.compile(optimizer='adam',loss = 'mean_squared_error',metrics=["accuracy"])
# model.fit(x_train,y_train,epochs=10)


# def check(num):
#     if num in [2,4,6,8,10,11,13,15,17,20,22,24,26,28,29,31,33,35]:
#         target = 'Black'
#     elif num in [1,3,5,7,9,12,14,16,18,19,21,23,25,27,30,32,34,36]:
#         target = 'Red'
#     elif num == 0:
#         target = 'Green'
        
#     if target == 'Red':
#         target_enc = [[0,0,1]]
#     elif target == 'Black':
#         target_enc = [[1,0,0]]
#     elif target == 'Green':
#         target_enc = [[0,1,0]]
    
    
#     prediction = model.predict(np.array([Data[-1]]))
#     prediction = prediction.tolist()
    
#     ind = prediction[0].index(max(prediction[0]))
#     for i in range(3):
#         prediction[0][i] = 0
#     prediction[0][ind] = 1

#     if prediction == [[1,0,0]]:
#         predicted_color = 'Black'
#     elif prediction == [[0,1,0]]:
#         predicted_color = 'Green'
#     elif prediction == [[0,0,1]]:
#         predicted_color = 'Red'
    
#     iscorrect = (predicted_color ==  target) 
#     print(iscorrect)
#     return predicted_color
        

        


AttributeError: module 'tensorflow' has no attribute 'get_default_graph'